In [1]:
import pandas as pd

def extraer_mensajes(csv_file, output_file):
    try:
        # Lee el archivo CSV
        df = pd.read_csv(csv_file)

        # Verifica si la columna 'Mensajes' está presente en el DataFrame
        if 'Mensaje' not in df.columns:
            raise ValueError("La columna 'Mensajes' no está presente en el archivo CSV.")

        # Extrae solo la columna 'Mensajes'
        mensajes_columna = df['Mensaje']

        # Guarda los mensajes en un archivo de texto
        mensajes_columna.to_csv(output_file, index=False, header=False)

        print(f"Se extrajeron los mensajes y se guardaron en {output_file}")

    except Exception as e:
        print(f"Error: {e}")

# Reemplaza 'archivo.csv' con tu archivo CSV de entrada y 'mensajes.txt' con el nombre deseado para el archivo de salida.
archivo_csv = 'filtered.csv'
archivo_salida_txt = 'mensajes.txt'

extraer_mensajes(archivo_csv, archivo_salida_txt)


Se extrajeron los mensajes y se guardaron en mensajes.txt


In [2]:
#read input file to inspect it
with open('input.txt', 'r', encoding='UTF-8') as f:
    text = f.read()

In [3]:
print('Length of dataset in characters: ', len(text))

Length of dataset in characters:  1532264


In [4]:
# watching all the unique characters that occur in this dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_`abcdefghijklmnopqrstuvwxyz{|~ ¡¥´¹¿ÁÉÍÑÓ×ÚÝáãçèéìíñòó÷úüģĪĶķĺŕź́̃Տسعىيᐯᑌᑎᑕᑭᖇᖴᗩᗪᗰᗴ‎’“”…‪‬ €
155


In [6]:
#Tokenize the imput text (trnasform each individual character (for this case) into integers using encoders and for vice versa using a decoder)
#create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) #decoder: taker a list of integers, output a string
print(encode("hii there"))
print(decode(encode("hii there")))

[72, 73, 73, 1, 84, 72, 69, 82, 69]
hii there


In [7]:
# Encoding the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1532264]) torch.int64
tensor([  3,  46,  69,  83,  83,  65,  71,  69,  83,   1,  65,  78,  68,   1,
         67,  65,  76,  76,  83,   1,  65,  82,  69,   1,  69,  78,  68,  14,
         84,  79,  14,  69,  78,  68,   1,  69,  78,  67,  82,  89,  80,  84,
         69,  68,  15,   1,  47,  79,   1,  79,  78,  69,   1,  79,  85,  84,
         83,  73,  68,  69,   1,  79,  70,   1,  84,  72,  73,  83,   1,  67,
         72,  65,  84,  13,   1,  78,  79,  84,   1,  69,  86,  69,  78,   1,
         56,  72,  65,  84,  83,  34,  80,  80,  13,   1,  67,  65,  78,   1,
         82,  69,  65,  68,   1,  79,  82,   1,  76,  73,  83,  84,  69,  78,
          1,  84,  79,   1,  84,  72,  69,  77,  15,   3,   0,  42,  86,  79,
         78,  78,  69,   1,  34,  76,  86,  65,  82,  65,  68,  79,   1,  67,
         82,  69,  65,  84,  69,  68,   1,  84,  72,  73,  83,   1,  71,  82,
         79,  85,  80,   0,  35,  85,  69,  78,  65,  83,   1,  84,  65,  82,
         68,  69,  83,   1,  6

In [8]:
#Spliting up data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
#Data loader (batches of chunk of data)
block_size = 8 #the context of words the model has to consider for inferences
train_data[:block_size+1]

tensor([ 3, 46, 69, 83, 83, 65, 71, 69, 83])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is: {context}, the target is: {target}")

When input is: tensor([3]), the target is: 46
When input is: tensor([ 3, 46]), the target is: 69
When input is: tensor([ 3, 46, 69]), the target is: 83
When input is: tensor([ 3, 46, 69, 83]), the target is: 83
When input is: tensor([ 3, 46, 69, 83, 83]), the target is: 65
When input is: tensor([ 3, 46, 69, 83, 83, 65]), the target is: 71
When input is: tensor([ 3, 46, 69, 83, 83, 65, 71]), the target is: 69
When input is: tensor([ 3, 46, 69, 83, 83, 65, 71, 69]), the target is: 83


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 1,  1, 38, 78,  1, 84, 79, 82],
        [77, 73, 78, 85, 84, 79, 83,  1],
        [71, 85, 83, 84, 65,  1, 80, 79],
        [69, 83, 67, 85, 66, 82, 73, 82]])
targets:
torch.Size([4, 8])
tensor([[ 1, 38, 78,  1, 84, 79, 82, 78],
        [73, 78, 85, 84, 79, 83,  1, 89],
        [85, 83, 84, 65,  1, 80, 79, 68],
        [83, 67, 85, 66, 82, 73, 82,  1]])
----
when input is [1] the target: 1
when input is [1, 1] the target: 38
when input is [1, 1, 38] the target: 78
when input is [1, 1, 38, 78] the target: 1
when input is [1, 1, 38, 78, 1] the target: 84
when input is [1, 1, 38, 78, 1, 84] the target: 79
when input is [1, 1, 38, 78, 1, 84, 79] the target: 82
when input is [1, 1, 38, 78, 1, 84, 79, 82] the target: 78
when input is [77] the target: 73
when input is [77, 73] the target: 78
when input is [77, 73, 78] the target: 85
when input is [77, 73, 78, 85] the target: 84
when input is [77, 73, 78, 85, 84] the target: 79
when input is [77, 73, 78, 85

In [12]:
print(xb) #input to the transformer

tensor([[ 1,  1, 38, 78,  1, 84, 79, 82],
        [77, 73, 78, 85, 84, 79, 83,  1],
        [71, 85, 83, 84, 65,  1, 80, 79],
        [69, 83, 67, 85, 66, 82, 73, 82]])


In [22]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

#Printing a generated text. This would be garbage 'cause I'm using seeds which randomize the outputs basically 
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
# Need more explaniation

torch.Size([32, 155])
tensor(5.5981, grad_fn=<NllLossBackward0>)

íg%÷ع$#LiÚᑎ)qãIÓĺ?1lb~*úO8¡8.ìSxÁlñx“Váhèź¡́1>r^ᗰWᑎc¡Ñᖴ@÷ÝÑKᗴ^=Í.¿rÍçU×ü´IYq|Zã@”ÚczQNىü×V=Kي=f¡/÷´ي


In [25]:
#Training the model to be less random
# for this porpuse it's recommendated to use a Pytorch optimization object
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [30]:
batch_size = 32
for steps in range(10000):    
    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3476738929748535


In [31]:
#Printing a generated text.
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
#Becasuse this model its only watching for the last word to generete other character the majority of them has nosense for the reason that they are not talking to 
#each other and figuring out what it's in the context 


Sara l siargasueren ra l s elia es de el Sicos cherelo e adon dijo má, pajost jZ+? ríaro s elmastio 
